# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui

sinasc.isna().sum().sort_values(ascending=False)



DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DTCADASTRO        0
NUMEROLOTE        0
VERSAOSIST        0
ORIGEM            0
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui

lista_variaveis = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

sinasc_v2 = sinasc[lista_variaveis]

sinasc_v2.isna().sum().sort_values(ascending = False)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [4]:
# 4) seu código aqui

sinasc_v2 = sinasc_v2[~sinasc_v2['APGAR5'].isna()]

print(sinasc_v2.shape)

print(sinasc_v2.isna().sum().sort_values(ascending=False))

(26925, 9)
QTDFILVIVO    1566
GESTACAO      1216
ESTCIVMAE      315
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
APGAR5           0
dtype: int64


In [5]:
# 5) seu código aqui

print(sinasc_v2['ESTCIVMAE'].isna().sum())

print(sinasc_v2['CONSULTAS'].isna().sum())



315
0


In [6]:
sinasc_v2_copy = sinasc_v2.copy()

sinasc_v2_copy['ESTCIVMAE'] = sinasc_v2_copy['ESTCIVMAE'].fillna(9)

print(sinasc_v2['ESTCIVMAE'].isna().sum())

315


In [7]:
# 6) Seu código aqui

sinasc_v2_copy['QTDFILVIVO'] = sinasc_v2_copy['QTDFILVIVO'].fillna(0)

sinasc_v2_copy['QTDFILVIVO'].isna().sum()


0

> Para a variável gravidez, optei pela moda, pois o dado "Única", representava 98% da base o que não trará grandes impactos em casos de analise estatistica

In [9]:
# 7) seu código aqui

sinasc_v2_copy['GRAVIDEZ'] = sinasc_v2_copy['GRAVIDEZ'].fillna(sinasc_v2_copy['GRAVIDEZ'].mode()[0])

sinasc_v2_copy['GRAVIDEZ'].isna().sum()


0

In [11]:
sinasc_v2_copy['GRAVIDEZ'].value_counts()

Única              26432
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [12]:
sinasc_v2_copy['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

> Para a variável "escolaridade da mãe", idetifiquei no dicionário a opção "9 - Ignorado" e neste caso optei por substituir por este dado, uma vez que as demais faixas tem uma quantidade consideravel e copialas poderia distorcer os resultados de analises. 


In [14]:
sinasc_v2_copy['ESCMAE'] = sinasc_v2_copy['ESCMAE'].fillna('Ignorado')

sinasc_v2_copy['ESCMAE'].isna().sum()

0

In [15]:
sinasc_v2_copy['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Ignorado             310
Nenhuma               40
Name: ESCMAE, dtype: int64

In [16]:
sinasc_v2_copy['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

> Para a variável "gestação", optei por utilizar o método ffill, para copiar o intem anterior. acredito que neste caso acontecerá uma distribuição aleatória, em uma quantidade pequena da base (4,5%)

In [19]:
sinasc_v2_copy['GESTACAO'] = sinasc_v2_copy['GESTACAO'].fillna(method='ffill')

sinasc_v2_copy['GESTACAO'].isna().sum()

0

In [20]:
sinasc_v2_copy['GESTACAO'].value_counts()

37 a 41 semanas        23570
32 a 36 semanas         2359
42 semanas e mais        724
28 a 31 semanas          174
22 a 27 semanas           89
Menos de 22 semanas        9
Name: GESTACAO, dtype: int64

In [21]:
# 8) seu código aqui

sinasc_v2_copy.loc[sinasc_v2_copy['APGAR5'] <= 3, 'APGAR5_CLASS'] = 'asfixia severa'
sinasc_v2_copy.loc[(sinasc_v2_copy['APGAR5'] > 3) & (sinasc_v2_copy['APGAR5'] <= 5),
           'APGAR5_CLASS'] = 'asfixia moderada'
sinasc_v2_copy.loc[(sinasc_v2_copy['APGAR5'] > 5) & (sinasc_v2_copy['APGAR5'] <= 7),
           'APGAR5_CLASS'] = 'asfixia leve'
sinasc_v2_copy.loc[(sinasc_v2_copy['APGAR5'] > 7), 'APGAR5_CLASS'] = 'normal' 

sinasc_v2_copy

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CLASS
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [23]:
sinasc_v2_copy.loc[sinasc_v2_copy['APGAR5'] < 3 ].head(3)                 
               

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CLASS
20,4,29,5.0,4 a 7 anos,3.0,32 a 36 semanas,Única,3,0.0,asfixia severa
1095,3,24,1.0,4 a 7 anos,3.0,28 a 31 semanas,Única,1,2.0,asfixia severa
1292,1,19,2.0,4 a 7 anos,0.0,22 a 27 semanas,Única,2,0.0,asfixia severa


In [24]:
sinasc_v2_copy.loc[(sinasc_v2_copy['APGAR5'] > 3) & (sinasc_v2_copy['APGAR5'] <= 5)].head(3)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CLASS
182,1,30,5.0,8 a 11 anos,2.0,22 a 27 semanas,Única,3,4.0,asfixia moderada
418,1,14,5.0,4 a 7 anos,0.0,37 a 41 semanas,Única,3,5.0,asfixia moderada
673,1,33,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,4.0,asfixia moderada


In [25]:
sinasc_v2_copy.loc[(sinasc_v2_copy['APGAR5'] > 5) & (sinasc_v2_copy['APGAR5'] <= 7)].head(3)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CLASS
14,1,31,1.0,4 a 7 anos,1.0,37 a 41 semanas,Única,4,7.0,asfixia leve
205,1,28,2.0,4 a 7 anos,2.0,37 a 41 semanas,Única,3,6.0,asfixia leve
243,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,7.0,asfixia leve


In [26]:
sinasc_v2_copy.loc[sinasc_v2_copy['APGAR5'] > 7 ].head(3)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CLASS
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal


In [27]:
# 9) seu código aqui

sinasc_v2_copy.columns


Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'APGAR5_CLASS'],
      dtype='object')

In [31]:
sinasc_v2_copy.columns = ['loc_nas', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao',
       'gravidez', 'consultas', 'apgar_5', 'apgar_5_class']

sinasc_v2_copy.columns

Index(['loc_nas', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar_5', 'apgar_5_class'],
      dtype='object')

In [32]:
sinasc_v2_copy.head(5)

,loc_nas,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,apgar_5_class
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
